# Add Methane into a Cluster
Take a water in the middle and make it methane!

In [1]:
from ase.io import read
from ase.db import connect
from ase.build import molecule
from random import sample
import numpy as np

Configuration

In [2]:
number_to_make = 128

## Load in Some Clusters
Get all of the clusters we used for training

In [3]:
with connect('../initial-database/initial-ttm.db') as db:
    water_clusters = []
    for row in db.select():
        atoms = row.toatoms()
        atoms.info['filename'] = row.filename
        water_clusters.append(atoms)
print(f'Loaded {len(water_clusters)} water clusters')

Loaded 1720 water clusters


## Make a Function to do the 
We're going to take a water in the middle and make it a methane

In [4]:
def replace_inner_water(atoms):
    """Replace the centermost water in a structure with a methane
    
    Args:
        atoms: Structure to alter
    Returns:
        A structure that now includes a water
    """
    
    # Create a copy
    atoms = atoms.copy()
    
    # Find the water closest to the center of mass
    center_O = np.linalg.norm(atoms.positions[::3] - atoms.get_center_of_mass(), axis=1).argmin()

    # Delete that water
    center_O *= 3 
    center_pos = atoms.positions[center_O]
    del atoms[center_O:center_O + 3]  # Next two atoms are its waters
    assert atoms.get_chemical_formula(empirical=True) == 'H2O'

    # Add in a methane at that position instead

    methane = molecule('CH4')
    methane.set_center_of_mass(center_pos)
    new_strc = atoms + methane
    new_strc.info = atoms.info

    # Quality check: No overlapping atoms
    dists = new_strc.get_all_distances()
    assert dists[np.triu_indices_from(dists, k=1)].min() > 0.5
    
    return new_strc

In [5]:
replace_inner_water(water_clusters[0])

Atoms(symbols='OH2OH2OH2OH2OH2OH2OH2OH2OH2OH2CH4', pbc=False)

## Make a bunch of them
Give our algorithms a good place to start

In [6]:
to_replace = sample(water_clusters, number_to_make)

In [7]:
with connect('methane-added.db', append=False) as db:
    for a in to_replace:
        atoms = replace_inner_water(a)
        db.write(atoms, **atoms.info)